<a href="https://colab.research.google.com/github/Keerthana-hack24/ACM-JUMP-PROJECT-KEERTHANA/blob/main/MOVIE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
credits=pd.read_csv('/content/credits.csv')
titles=pd.read_csv('/content/titles.csv')


In [4]:
credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [5]:
titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [6]:
#selecting the important features for further analysis
selected_features = ['genres','description','release_year','imdb_score','age_certification','role','character']
print(selected_features)

['genres', 'description', 'release_year', 'imdb_score', 'age_certification', 'role', 'character']


In [7]:
for feature in selected_features:
    if feature in credits.columns:
        credits[feature] = credits[feature].fillna('')
    if feature in titles.columns:
        titles[feature] = titles[feature].fillna('')

In [8]:
final = pd.merge(
    titles,
    credits,
    left_on='id',
    right_on='id',
    how='left'  # Keeps all titles even if no credits exist
)

In [9]:
final.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,,NaN,0.600,NaN,NaN,NaN,NaN,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3748.0,Robert De Niro,Travis Bickle,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,14658.0,Jodie Foster,Iris Steensma,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,7064.0,Albert Brooks,Tom,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3739.0,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR


In [10]:
combined_features = (
    final['genres'].fillna('').astype(str) + ' ' +
    final['description'].fillna('').astype(str) + ' ' +
    final['release_year'].fillna('').astype(str) + ' ' +
    final['imdb_score'].fillna('').astype(str) + ' ' +
    final['age_certification'].fillna('').astype(str) + ' ' +
    final['role'].fillna('').astype(str)+' '+
    final['character'].fillna('').astype(str)
)
combined_features

,0
0,['documentation'] This collection includes 12 ...
1,"['drama', 'crime'] A mentally unstable Vietnam..."
2,"['drama', 'crime'] A mentally unstable Vietnam..."
3,"['drama', 'crime'] A mentally unstable Vietnam..."
4,"['drama', 'crime'] A mentally unstable Vietnam..."
...,...
78157,['comedy'] A controversial TV host and comedia...
78158,['comedy'] A controversial TV host and comedia...
78159,['comedy'] A controversial TV host and comedia...
78160,"[] Jamie Foxx, David Alan Grier and more from ..."


In [11]:
vectorizer = TfidfVectorizer()

In [12]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [13]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3119562 stored elements and shape (78162, 42925)>
  Coords	Values
  (0, 10522)	0.10286132234680322
  (0, 38182)	0.10021199982419404
  (0, 7731)	0.2552694008564709
  (0, 17903)	0.2169431761723191
  (0, 33)	0.17945672528987777
  (0, 41555)	0.09656901021148231
  (0, 40846)	0.11241289679775045
  (0, 17640)	0.17646717541328688
  (0, 12024)	0.19088944335764282
  (0, 30081)	0.34442165142727266
  (0, 13203)	0.21067422293271182
  (0, 23429)	0.1553430619700372
  (0, 27278)	0.0432838551165772
  (0, 41171)	0.12498251865032481
  (0, 2325)	0.0958341641758942
  (0, 15236)	0.34442165142727266
  (0, 1772)	0.041006533829334384
  (0, 27288)	0.31172722178676754
  (0, 10321)	0.34442165142727266
  (0, 17863)	0.0483294808140514
  (0, 38093)	0.03744829721344471
  (0, 10527)	0.21906977724368523
  (0, 13317)	0.150788895289041
  (0, 5991)	0.20194708277972248
  (0, 3237)	0.12066976263056788
  :	:
  (78161, 17863)	0.1030964597798956
  (78161, 38093)	0.0

In [1]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

NameError: name 'cosine_similarity' is not defined